# Quantum Computing with ndarray

This notebook contains exercises that will guide you into implementing basic quantum computing concepts, and essential linear algebra operations. We have defined for you a simple `ndlist` class in `ndlists.py` that you will extend with methods to manipulate quantum states and operators. The goal is to get familiar with the mathematical framework of quantum computing without using specialized libraries like numpy.

This first notebook focuses on the representation of quantum states (kets and bras), measurement probabilities, unitary operators, and inner products. By the end of this first notebook, you should have defined the scalar product and understood the concept of base, superposition, and measurement in quantum mechanics.

### At the end of each session you should have added all your functions from this notebook to a separated file called "TME1_name1_name2.py" where name1 and name 2 are the names of your pair. If you fail to properly rename that file, and add the correct name you will be penalized.

In [100]:
from ndlists import ndlist
from typing import List

from math import cos, sin, sqrt, pi

## Exercise 0 — Floating Point Precision

The floating point representation of numbers in computers is not exact. This can lead to unexpected results when comparing floating point numbers directly. For example:

In [101]:
a = 0.1 + 0.2
b = 0.3
print(a == b)  # This will print False

epsilon = 1e-15 # try 1e-15
print(1 + epsilon -1)

epsilon = 1e-16
print(1 + epsilon -1)

False
1.1102230246251565e-15
0.0


To avoid such issues, we can define a method to compare floating point numbers with a tolerance level.

In [102]:
def _isclose(a: float, b: float, tol: float = 1e-9) -> bool:
    """
    Check if two floating point numbers are close to each other within a tolerance.
    :param a: first floating point number
    :param b: second floating point number
    :param tol: tolerance level
    :return: True if numbers are close, False otherwise
    """
    return abs(a - b) < tol

In [103]:
# Test the _isclose method
print(_isclose(a, b))  # This should print True

True


## Exercise 1 — Ket
A **ket** represents a column vector describing the state of a quantum system.
For example, the computational basis states are

$$
|0⟩ = \begin{bmatrix} 1 \\ 0 \end{bmatrix}, \quad |1⟩ = \begin{bmatrix} 0 \\ 1 \end{bmatrix}
$$

Within the programming framework, we will represent a ket as a ndarray object of shape $(n, 1)$, where $n > 1$, to avoid ambiguity. Is a (1,1) array a ket or a bra, or a vector at all?

**Tasks:**
1. Implement a method `_ket(lst: list)` that constructs a ket from a list.
2. Test your method with |0⟩ and |1⟩.

In [104]:
def _ket(lst: list) -> 'ndlist':
    """
    Create a ket from a list.
    :param lst: list of elements
    :return: ndlist representing the ket
    """
    # A ket is a ndarray object of shape $(n, 1)$, where $n > 1$, to avoid ambiguity
    assert len(lst) > 1, "A ket must have more than one element"
    # Assert that it is not a list of lists for example [[0,1]] is not a ket
    assert not any(isinstance(item, list) for item in lst), "A ket must not be a list of lists"    
    return ndlist([[x] for x in lst])

In [105]:
# Test your method
zero = _ket([1, 0])
one = _ket([0, 1])

print("|0⟩:", zero, zero.shape)
print("|1⟩:", one, one.shape)

|0⟩: [[1], [0]] (2, 1)
|1⟩: [[0], [1]] (2, 1)


Let us now define two important operations when manipulating quantum states, namely the norm and the normalization of a state. The norm of a vector $v$ is defined as $\|v\| = \sqrt{\sum_i |v_i|^2}$.

**Tasks:**

1. Implement a method `_norm(array: ndlist)` that computes the norm of a ndlist object.
2. Write a function that multiply a ndlist object by a scalar called `_scalar_mult(scalar: float, array: ndlist)`.
3. Implement a method that normalizes a ndlist object.

In [106]:
def _norm(array: ndlist) -> float:
    """
    Compute the norm of a ndlist object.
    :param array: ndlist object
    :return: norm (float)
    """
    return sqrt(sum(x[0]**2 for x in array))    

In [107]:
def _scalar_mult(scalar: complex, array: ndlist) -> 'ndlist':
    """
    Multiply a ndlist object by a scalar.
    :param scalar: scalar
    :param array: ndlist
    :return: ndlist object after multiplication
    """
    # Support vectors and matrices; multiply element-wise and preserve shape
    if len(array) > 0 and isinstance(array[0], list):
        return ndlist([[scalar * array[i][j] for j in range(len(array[0]))] for i in range(len(array))])
    return ndlist([scalar * element for element in array])

In [108]:
def _normalize(array: ndlist) -> 'ndlist':
    """
    Normalize a ndlist object.
    :param array: ndlist object
    :return: normalized ndlist object
    """
    # Check for 0
    norm = _norm(array)
    if norm == 0:
        raise ValueError("Cannot normalize a zero vector.")
    # Use _norm and _scalar_mult to normalize a ndlist object.
    return _scalar_mult(1/_norm(array), array)

In [109]:
# Test your methods
print("Norm of |0⟩:", _norm(zero))  # Should be 1
print("Norm of |1⟩:", _norm(one))   # Should be 1

superposition = _ket([1, 1])
print("Norm of superposition |+⟩:", _norm(superposition))  # Should be sqrt(2)

normalized_superposition = _normalize(superposition)
print("Normalized superposition |+⟩:", normalized_superposition, _norm(normalized_superposition))  # Should be 1

Norm of |0⟩: 1.0
Norm of |1⟩: 1.0
Norm of superposition |+⟩: 1.4142135623730951
Normalized superposition |+⟩: [[0.7071067811865475], [0.7071067811865475]] 0.9999999999999999


## Exercise 2 - Matrix representation

A matrix can represent a quantum operator. For example, the Pauli matrix $\sigma_x$ is represented by the matrix

$$
\sigma_x = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}
$$

A matrix is a 2D ndarray object, with shape $(n, m)$, with $n, m > 1$, to avoid ambiguity of vector with shape (1, n) or (n, 1), and (1, 1). A matrix is defined in a basis, meaning it can be decomposed as:

$$
A = \sum_{i,j} A_{ij} |i⟩⟨j|
$$

**Tasks:**
1. Implement a function `_zeros(n: int, m: int)` that creates a zero matrix of shape (n, m).
2. Implement a function `_identity(n: int)` that creates an identity matrix of shape (n, n).
3. Implement a method `_matrix(lst: list)` that constructs a matrix from a list of kets.
4. Define the following operations:
    - Matrix multiplication `_matmul`
    - Determinant `_det`
    - Transpose `_transpose` and Hermitian conjugate `_hermitian`

In [110]:
def _zeros(n: int, m: int) -> 'ndlist':
    """
    Create a zero matrix of shape (n, m).
    :param n: number of rows
    :param m: number of columns
    :return: ndlist representing the zero matrix
    """
    return ndlist([[0 for _ in range(m)] for _ in range(n)])

In [111]:
def _identity(n: int) -> 'ndlist':
    """
    Create an identity matrix of shape (n, n).
    :param n: size of the identity matrix
    :return: ndlist representing the identity matrix
    """
    return ndlist([[1 if i == j else 0 for j in range(n)] for i in range(n)])

In [112]:
# Create a zero matrix
zero_matrix = _zeros(2, 2)
print("Zero matrix:\n", zero_matrix, zero_matrix.shape)

# Create an identity matrix
identity_matrix = _identity(2)
print("Identity matrix:\n", identity_matrix, identity_matrix.shape)

Zero matrix:
 [[0, 0], [0, 0]] (2, 2)
Identity matrix:
 [[1, 0], [0, 1]] (2, 2)


In [113]:
def _matrix(lvec: List[ndlist]) -> 'ndlist':
    """
    Create a matrix from a list of lists using the zero matrix function.
    :param lvec: list of lists representing the matrix
    :return: ndlist representing the matrix
    """
    if not lvec:
        raise ValueError("The list of vectors is empty.")
    n = len(lvec)
    m = len(lvec[0])
    for vec in lvec:
        if len(vec) != m:
            raise ValueError("All vectors must have the same length.")
    mat = _zeros(n, m)
    for i in range(n):
        for j in range(m):
            mat[i, j] = lvec[i][j, 0]
    return mat

In [114]:
# Create a matrix from a list of ndlist
matrix_A = _matrix([_ket([1, 0]), _ket([0, 1])])
print("Matrix A:\n", matrix_A, matrix_A.shape)

Matrix A:
 [[1, 0], [0, 1]] (2, 2)


In [115]:
def _matmul(A: ndlist, B: ndlist) -> 'ndlist':
    """
    Perform matrix multiplication A @ B.
    :param A: ndlist representing matrix A
    :param B: ndlist representing matrix B
    :return: ndlist representing the result of A @ B
    """
    if A.shape[1] != B.shape[0]:
        raise ValueError("Incompatible shapes for matrix multiplication")

    result = _zeros(A.shape[0], B.shape[1])

    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            sum_product = 0
            for k in range(A.shape[1]):
                sum_product += A[i, k] * B[k, j]
            result[i, j] = sum_product

    return result

In [116]:
# Matrix multiplication
A = ndlist([[1, 2], [3, 4]])
B = ndlist([[5, 6], [7, 8]])

C = _matmul(A, B)
print("Matrix multiplication A @ B:\n", C)
A_identity = _matmul(A, _identity(2))
print("A @ I:\n", A_identity)

Matrix multiplication A @ B:
 [[19, 22], [43, 50]]
A @ I:
 [[1, 2], [3, 4]]


In [117]:
def _det(matrix: ndlist) -> float:
    """
    Compute the determinant of a square matrix.
    :param matrix: ndlist representing a square matrix
    :return: determinant (float)
    """
    # Recursive approach to compute the determinant of a square matrix.
    if matrix.shape[0] != matrix.shape[1]:
        raise ValueError("Matrix must be square to compute determinant.")

    # Base case for 2x2 matrix
    if matrix.shape == (2, 2):
        return matrix[0, 0] * matrix[1, 1] - matrix[0, 1] * matrix[1, 0]

    # Recursive case
    determinant = 0
    for col in range(matrix.shape[1]):
        # Create the minor matrix by excluding the first row and current column
        minor = ndlist([
            [matrix[i, j] for j in range(matrix.shape[1]) if j != col]
            for i in range(1, matrix.shape[0])
        ])
        # Add or subtract the cofactor
        determinant += ((-1) ** col) * matrix[0, col] * _det(minor)

    return determinant

In [118]:
# Determinant
det_A = _det(A)
det_identity = _det(identity_matrix) # Should be 1

print("Determinant of A:", det_A)
print("Determinant of Identity matrix:", det_identity)

Determinant of A: -2
Determinant of Identity matrix: 1


In [119]:
def _transpose(matrix: ndlist) -> 'ndlist':
    """
    Compute the transpose of a matrix.
    :param matrix: ndlist representing a matrix
    :return: ndlist representing the transposed matrix
    """
    n, m = matrix.shape
    transposed = _zeros(m, n)
    for i in range(n):
        for j in range(m):
            transposed[j, i] = matrix[i, j]
    return transposed

In [120]:
def _hermitian(matrix: ndlist) -> 'ndlist':
    """
    Compute the Hermitian conjugate (conjugate transpose) of a matrix.
    :param matrix: ndlist representing a matrix
    :return: ndlist representing the Hermitian conjugate
    """
    n, m = matrix.shape
    hermitian = _zeros(m, n)
    for i in range(n):
        for j in range(m):
            hermitian[j, i] = complex(matrix[i, j]).conjugate()
    return hermitian

In [121]:
# Transpose
transpose_A = _transpose(A)
transpose_B = _transpose(B)

print("Transpose of A:\n", transpose_A)
print("Transpose of B:\n", transpose_B)

# Hermitian (for real matrices, it's the same as transpose)
complex_matrix = ndlist([[1+1j, 2], [3, 4-1j]])
hermitian_complex = _hermitian(complex_matrix)
print("Hermitian of complex matrix:\n", hermitian_complex)

Transpose of A:
 [[1, 3], [2, 4]]
Transpose of B:
 [[5, 7], [6, 8]]
Hermitian of complex matrix:
 [[(1-1j), (3-0j)], [(2-0j), (4+1j)]]


## Exercise 3 — Unitary Operators
A matrix \(U\) is **unitary** if

$$
U^\dagger U = I
$$

where $U^\dagger$ is the conjugate transpose of $U$ and $I$ is the identity matrix.

**Tasks:**
1. Implement a method `is_unitary()` that checks whether a given operator is unitary.
2. Implement an `apply_unitary(U)` method that applies a unitary matrix $U$ to a ket.
3. Show with an example:
   - Start with a ket (e.g., |0⟩).
   - Apply a quantum gate (e.g., Pauli-X).
   - Measure probabilities.
4. Prove/verify: applying $U$ then measuring is equivalent to measuring in the rotated basis.


In [122]:
def is_unitary(matrix: ndlist, tol: float = 1e-9) -> bool:
    """
    Check if a matrix is unitary.
    :param matrix: ndlist object representing a matrix
    :param tol: tolerance level for floating point comparison
    :return: True if unitary, False otherwise
    """
    if matrix.shape[0] != matrix.shape[1]:
        return False
    identity = _identity(matrix.shape[0])
    product = _matmul(matrix, _hermitian(matrix))
    for i in range(identity.shape[0]):
        for j in range(identity.shape[1]):
            if abs(product[i, j] - identity[i, j]) > tol:
                return False
    return True

In [123]:
# Define Pauli-X gate
pauli_x = ndlist([[0, 1], [1, 0]])
print("Is Pauli-X unitary?", is_unitary(pauli_x))  # Should be True

# Hadamard gate
hadamard = _scalar_mult(1/sqrt(2), ndlist([[1, 1], [1, -1]]))
print("Is Hadamard unitary?", is_unitary(hadamard))  # Should be True

# Some matrix
some_matrix = ndlist([[1, 2], [3, 4]])
print("Is some_matrix unitary?", is_unitary(some_matrix))  # Should be False

Is Pauli-X unitary? True
Is Hadamard unitary? True
Is some_matrix unitary? False


In [124]:
def apply_unitary(ket: ndlist, U: ndlist) -> 'ndlist':
    """
    Apply a unitary matrix U to a ket.
    :param ket: ndlist representing the ket
    :param U: ndlist representing the unitary matrix
    :return: ndlist representing the new ket
    """
    # Check if the unitary matrix is unitary.
    if not is_unitary(U):
        raise ValueError("The unitary matrix is not unitary.")
    # Check if the shapes of the ket and the unitary matrix are compatible for multiplication.
    if U.shape[1] != ket.shape[0]:
        raise ValueError("Incompatible shapes for matrix multiplication.")
    # Multiply the ket by the unitary matrix.
    return _matmul(U, ket)

In [125]:
# Apply Pauli-X to |0⟩
new_state = apply_unitary(zero, pauli_x)
print("New state after applying Pauli-X to |0⟩:\n", new_state)

# Hadamard gate
new_state_h = apply_unitary(zero, hadamard)
print("New state after applying Hadamard to |0⟩:\n", new_state_h)

New state after applying Pauli-X to |0⟩:
 [[0], [1]]
New state after applying Hadamard to |0⟩:
 [[0.7071067811865475], [0.7071067811865475]]


## Exercise 4 — Bra
A **bra** is the Hermitian conjugate (transpose + complex conjugate) of a ket:

$$
⟨ψ| = (|ψ⟩)^\dagger
$$

**Tasks:**
1. Implement a method `bra(ket: ndlist)` returning the bra associated to a ket.
2. Verify for |0⟩ and |1⟩.

In [126]:
def bra(ket: ndlist) -> 'ndlist':
    """
    Compute the bra associated to a ket.
    :param ket: ndlist representing the ket
    :return: ndlist representing the bra
    """
    # The bra is the Hermitian conjugate of the ket.
    return _hermitian(ket)

In [127]:
# Test bra method
bra_zero = bra(zero)
bra_one = bra(one)

print("Bra of |0⟩:", bra_zero, bra_zero.shape)
print("Bra of |1⟩:", bra_one, bra_one.shape)

Bra of |0⟩: [[(1-0j), -0j]] (1, 2)
Bra of |1⟩: [[-0j, (1-0j)]] (1, 2)


## Exercise 5 — Scalar Product
The scalar product between two states is

$$
⟨φ|ψ⟩ = \sum_i φ_i^* ψ_i
$$

**Tasks:**
1. Implement a method `_inner(ket1: ndlist, ket2: ndlist)` that computes the inner product between two kets.
2. Test your method with the provided examples.
3. Verify orthogonality and normalization of the states |0⟩, |1⟩, |+⟩, and |−⟩.


In [128]:
def _inner(ket1: ndlist, ket2: ndlist) -> complex:
    """
    Compute the inner product between two kets.
    :param ket1: ndlist representing the first ket
    :param ket2: ndlist representing the second ket
    :return: inner product (complex)
    """
    # The inner product is the sum of the products of the elements of the two kets.
    if ket1.shape != ket2.shape:
        raise ValueError("Kets must have the same shape for inner product.")
    bra1 = bra(ket1)
    product = _matmul(bra1, ket2)
    return product[0, 0]

In [129]:
# Test inner product method
plus = _normalize(_ket([1, 1]))  # |+⟩
minus = _normalize(_ket([1, -1]))  # |−⟩

print("Inner product ⟨0|1⟩:", _inner(zero, one))  # Should be 0
print("Inner product ⟨1|0⟩:", _inner(one, zero))  # Should be 0
print("Inner product ⟨+|+⟩:", _inner(plus, plus))  # Should be 1
print("Inner product ⟨−|−⟩:", _inner(minus, minus))  # Should be 1

Inner product ⟨0|1⟩: 0j
Inner product ⟨1|0⟩: 0j
Inner product ⟨+|+⟩: (0.9999999999999998+0j)
Inner product ⟨−|−⟩: (0.9999999999999998+0j)


# Exercise 6 : Measurement in an arbitrary basis

The goal is of this final exercise is to apply all the concepts we have seen so far to implement measurement in an arbitrary basis.
First you will define a matrix that is the decomposition in the basis of choice. Using the scalar product, you will compute the probability of measuring each state in that basis.

The probability of measuring outcome *i* is

$$
P_i = |a_i|^2
$$

for a state

$$
|ψ⟩ = \sum_i a_i |i⟩
$$

We will also see a key concept in quantum mechanics: global phase invariance.
A quantum state |ψ⟩ and the state exp(iφ)|ψ⟩ (where φ is a real number) represent the same physical state, as they yield identical measurement probabilities.



**Tasks:**
1. Implement a method `measure_in_basis(ket: ndlist, basis: List[ndlist])` that computes the probabilities of measuring the ket in the given basis.
2. Test your method with the provided examples.

In [130]:
def measure_in_basis(ket: ndlist, basis: ndlist) -> List[float]:
    """
    Compute the probabilities of measuring the ket in the given basis.
    :param ket: ndlist representing the ket
    :param basis: ndlist representing the basis as a matrix
    :return: list of probabilities
    """
    if basis.shape[0] != ket.shape[0]:
        raise ValueError("Basis and ket must have compatible shapes.")
    probabilities = []
    for i in range(basis.shape[1]):
        basis_vector = ndlist([[basis[j, i]] for j in range(basis.shape[0])])
        prob = abs(_inner(basis_vector, ket))**2
        probabilities.append(prob)
    total_prob = sum(probabilities)
    if total_prob == 0:
        raise ValueError("Total probability is zero, cannot normalize.")
    return [p / total_prob for p in probabilities]


In [131]:
# Define an arbitrary basis
theta = pi/4
basis = _matrix([_ket([cos(theta), sin(theta)]), _ket([cos(theta), -sin(theta)])])
print(basis, basis.shape)

[[0.7071067811865476, 0.7071067811865476], [0.7071067811865476, -0.7071067811865476]] (2, 2)


In [132]:
# Test measure_in_basis method with previously defined states
probabilities_zero = measure_in_basis(zero, basis)
probabilities_one = measure_in_basis(one, basis)
probabilities_plus = measure_in_basis(plus, basis)
probabilities_minus = measure_in_basis(minus, basis)

print("Probabilities of measuring |0⟩ in the basis:", probabilities_zero)
print("Probabilities of measuring |1⟩ in the basis:", probabilities_one)
print("Probabilities of measuring |+⟩ in the basis:", probabilities_plus)
print("Probabilities of measuring |−⟩ in the basis:", probabilities_minus)

Probabilities of measuring |0⟩ in the basis: [0.5, 0.5]
Probabilities of measuring |1⟩ in the basis: [0.5, 0.5]
Probabilities of measuring |+⟩ in the basis: [1.0, 0.0]
Probabilities of measuring |−⟩ in the basis: [0.0, 1.0]


In [133]:
# Test global phase invariance
phi = pi / 3  # Example phase
global_phase_state = _scalar_mult(cos(phi) + 1j * sin(phi), plus)
probabilities_global_phase = measure_in_basis(global_phase_state, basis)
print("Probabilities of measuring exp(iφ)|+⟩ in the basis:", probabilities_global_phase)

Probabilities of measuring exp(iφ)|+⟩ in the basis: [1.0, 0.0]


Define the following matrices and check their properties, especially in terms of phase invariance when applied to a ket:

1. Pauli-Z
2. Identity
3. Negative Identity

In [134]:
# Difference between pauli-z, identity and -identity in terms of phase invariance
# [[1, 0],[0, -1]]
pauli_z = ndlist([[1, 0],[0, -1]])
identity = ndlist([[1, 0],[0, 1]])
neg_identity = ndlist([[-1, 0],[0, -1]])

In [135]:
psi = _scalar_mult(1/sqrt(2), _ket([1, 1]))  # |+⟩ state
print("State |ψ⟩:", psi)

# Apply Pauli-Z
psi_pauli_z = apply_unitary(psi, pauli_z)
print("After applying Pauli-Z:", psi_pauli_z)
print("Probabilities after Pauli-Z:", measure_in_basis(psi_pauli_z, basis))
# Apply Identity
psi_identity = apply_unitary(psi, identity)
print("After applying Identity:", psi_identity)
print("Probabilities after Identity:", measure_in_basis(psi_identity, basis))
# Apply Negative Identity
psi_neg_identity = apply_unitary(psi, neg_identity)
print("After applying Negative Identity:", psi_neg_identity)
print("Probabilities after Negative Identity:", measure_in_basis(psi_neg_identity, basis))

State |ψ⟩: [[0.7071067811865475], [0.7071067811865475]]
After applying Pauli-Z: [[0.7071067811865475], [-0.7071067811865475]]
Probabilities after Pauli-Z: [0.0, 1.0]
After applying Identity: [[0.7071067811865475], [0.7071067811865475]]
Probabilities after Identity: [1.0, 0.0]
After applying Negative Identity: [[-0.7071067811865475], [-0.7071067811865475]]
Probabilities after Negative Identity: [1.0, 0.0]
